In [22]:
from __future__ import print_function
import numpy as np # Biblioteca de funciones matematicas de alto nivel
np.random.seed(1337)  # for reproducibility
import tensorflow as tf
import keras
import pandas
import time #Calcular tiempo medio clasificación
import os #Biblioteca para ejecutar comandos en el terminal
from ipaddress import ip_address    #Comparar distintas IPs
from ipaddress import ip_network    #Comparar distintas IPs
from datetime import datetime                       #Para convertir la fecha
from keras.preprocessing import sequence
from keras.models import Sequential # necesario para poder generar la red neuronal
from keras.layers import Dense, Dropout, Activation, Lambda # Tipos de capa, hacen lo siguiente:
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten, LSTM
from keras.callbacks import CSVLogger # para guardar los datos en un excel
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.datasets import imdb # un dataset incluido en keras
from keras import backend as K # importas el backend (Tensorflow, Theano, etc)
import pandas as pd # pandas es una libreria extension de numpy usada para manipulacion y analisis de datos, para manipular tablas numericas y series temporales
from keras.utils.np_utils import to_categorical # sirve para convertir vectores de enteros a una matriz de clases binaria
import h5py # para almacenar un monton de datos numericos y dar facilidades de manipulacion para datos de Numpy
from sklearn.preprocessing import Normalizer # Para normalizar los datos
from sklearn.model_selection import train_test_split #para hacer la separacion entre datos de test y train
from sklearn.preprocessing import OneHotEncoder #para convertir los datos de entrada
from sklearn.preprocessing import LabelEncoder #Para pasar a numeros

In [23]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

/opt/conda/lib/python3.8/site-packages/tensorflow/python/client/session.py:1761: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [24]:
##Incio función red neuronal
def red_neuronal():
    #Cargamos las etiquetas que son todos los ataques posibles que detecta nuestra red
    Labels =['Benign',
             'DoS attacks-GoldenEye',
             'DoS attacks-Hulk',
             'DoS attacks-SlowHTTPTest',
             'DoS attacks-Slowloris',
             'FTP-BruteForce',
             'SSH-Bruteforce']
             
    #creamos la red neuronal
    model = Sequential()
    model.add(Convolution1D(64, 3, activation="relu",input_shape=(77, 1), padding = 'same'))
    model.add(Convolution1D(64, 3, activation="relu", padding = 'same'))
    model.add(MaxPooling1D(pool_size=(2))) # capa donde se lleva a cabo el pooling, se queda con el maximo de cada 2
    model.add(Convolution1D(128, 3,  activation="relu", padding = 'same'))
    model.add(Convolution1D(128, 3,  activation="relu", padding = 'same'))
    model.add(MaxPooling1D(pool_size=(2)))
    model.add(Flatten()) #Si eliminamos red LSTM
    model.add(Dropout(0.1)) #
    model.add(Dense(len(Labels), activation="softmax")) # capa fully conected para decision final, usamos softmax porque con ella los valores finales tienen mas relacion con los valores
    # anteriores y no solo con 1

    # define optimizer and objective, compile cnn
    model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['categorical_accuracy']) # se compila la red neuronal con los siguientes parametros:

    #Cargamos los resultados obtenidos en la etapa de train
    model.load_weights("ResultadosRedes/saved_model_sin.hdf5")
    
    return model 
    ##Fin función red neuronal

In [25]:
#Inicio tratamiento de datos
def clasificador(dataset,model):

    #Configuración para que no salte warning falso positivo python pandas
    pd.options.mode.chained_assignment = None  # default='warn'
    
    #Nos quedamos con la quintupla para poder generar reglas
    quintupla = dataset.loc[:,['Src IP', 'Src Port', 'Dst IP','Dst Port','Protocol']]
    
    #Eliminamos los datos mal introducidos
    dataset = dataset.drop(dataset[dataset['Dst Port']=='Dst Port'].index)
    #Eliminamos las columnas innecesarias
    dataset = dataset.drop(['Flow ID', 'Src IP', 'Src Port', 'Dst IP','Dst Port'], axis=1)
    #Quitamos direcciones IP dest y src para no introducir sesgos, lo mismo con los puertos
    #Quitar el puerto de destino es una modificación con respecto al trabajo original
    #Eliminamos la columna de timestamp para no tener sesgo
    dataset = dataset.drop(['Timestamp'], axis=1)

    #Convertimos los datos a float64 para poder trabajar con ellos (menos el campo de etiquetas)
    n=0
    for column in dataset:
        column
        if column != 'Label':
            dataset[column] = dataset[column].astype(float)
    Y = dataset["Label"]
            
    # Replacing infinite and nan para evitar errores
    dataset.replace([np.inf, -np.inf], -1, inplace=True) 
    dataset.replace([np.nan, -np.nan], -1, inplace=True)

    #Establecemos nuestra entrada que vamos a testear
    #Todo menos las etiquetas
    X=dataset.iloc[:, 0:77]

    #Cargamos las etiquetas que son todos los ataques posibles que detecta nuestra red
    Labels =['Benign',
             'DoS attacks-GoldenEye',
             'DoS attacks-Hulk',
             'DoS attacks-SlowHTTPTest',
             'DoS attacks-Slowloris',
             'FTP-BruteForce',
             'SSH-Bruteforce']

    from sklearn.preprocessing import StandardScaler

    scaler = StandardScaler().fit(X) # Normalizamos los datos
    testT = scaler.transform(X)# Asi se representan los datos

    # reshape input to be [samples, time steps, features]
    X_test = np.reshape(testT, (testT.shape[0],testT.shape[1],1))
    
    #Cargamos los resultados obtenidos en la etapa de train
    model.load_weights("ResultadosRedes/saved_model_sin.hdf5")

    #predecimos la naturaleza de los paquetes de test
    #y_pred = (model.predict(X_test) > 0.5).astype("int32")     #Para clase binaria
    y_pred=np.argmax(model.predict(X_test), axis=-1)            #Para multiclase
    y_pred_etiq=y_pred;
    
    from sklearn.preprocessing import LabelEncoder #Para pasar a numeros
    lb_make = LabelEncoder()
    lb_make.fit_transform(Labels)
    y_pred_etiq=lb_make.inverse_transform(y_pred)
    
    #Configuración para que funciones servicio de firewall de Windows
    # os.system('sc config mpssvc start=auto')
    # os.system('net stop mpssvc && net start mpssvc')
    # os.system('netsh advfirewall set allprofiles state on')
    
    command='netsh advfirewall firewall add rule name = "BLACKLIST" dir=in action=block remoteip='  #Comando que bloquea IP en windows
    localIP='192.168.64.131'     #Por si quisiesemos comparar con una sola IP directamente     
    localIPs='192.168.64.131/32' #El paramétro 32 es modificable para casos generales
    lista_ips=[]                 #Lista que usaremos para bloquear la IP a la segunda 


    for index, value in np.ndenumerate(y_pred):         #Recorremos el array donde tenemos los resultados de la clasificación
        
    
        if y_pred[index]!=0:                           #Si el valor es distinto de 0 (benigno), entonces es un ataque y entramos
          # print(index[0],y_pred_etiq[index])           #Mostramos por pantalla el índice y el tipo de ataque detectado
          # print(quintupla['Src IP'][index[0]])         #Imprimimos la dirección IP del posible atacante
           
          if ip_address(quintupla['Src IP'][index[0]]) not in ip_network(localIPs): #Si la dirección atacante no está dentro de la red local entonces se bloquea
           
            if quintupla['Src IP'][index[0]] in lista_ips:  #Si la dirección del atacante se ha visto previamente (se ha guardado en lista_ips) bloqueamos :
                attackIP=quintupla['Src IP'][index[0]]          #Guardamos en la dirección IP del atacante en una variable auxiliar
                command=command+attackIP                        #Construimos el comando para bloquear la IP del atacante
                # os.system(command)                            #Ejecutamos el comando para bloquear la dirección IP del atacante
                #print(command)                                  #Imprimimos como sería comando para verlo ya que no lo ejecutamos por seguridad
                command='netsh advfirewall firewall add rule name = "BLACKLIST" dir=in action=block remoteip='  #Finalmente limpiamos el comando, para poder seguir usandolo con otras IPs
                
            else:                                           #Si no está en la lista_ips es que no la habíamos visto antes y entonces no se bloquea 
                lista_ips.append(quintupla['Src IP'][index[0]]) #Pero se añade a la lista para bloquearla si la volvemos a ver
  

        
#Hacer las dos medidas de ancho de banda sin elastic y con elastic pasandole un dataset grande para ver comportamiento real 

In [26]:
import csv
import time

In [27]:
#Declaración de variables auxiliares
filePath='Datasets/BW_prueba.csv'
# filePath='E:/TFG/CICFlowMeter/CICFlowMeter-master/build/install/CICFlowMeter/bin/data/daily/2022-02-14_Flow.csv'
dataset_size=1
t_total=0

#Construcción red neuronal y la guardamos en model 
model=red_neuronal()

#Abrimos el archivo y lo guardamos en una variable llamada f
with open(filePath) as f:                                          #Bucle que se ejecuta constantemente
    dataset=pd.read_csv(filePath, encoding = "ISO-8859-1")  #pd.read_csv lee el archivo csv entero  
    if dataset.shape[0]>dataset_size:                       #Si el número de filas del archivo leido (nº flujos) es mayor que el nº filas anterior entonces clasifico sino no 
        t_inicio=time.time()                                #Hallamos el tiempo antes de la clasificación
        clasificador(dataset,model)                       #Línea que se encarga de llamar a la red neuronal para clasificar nuevos flujos
        t_fin=time.time()                                   #Hallamos el tiempo después de la clasificación
        t_total=(t_fin-t_inicio)+t_total;                     #Hallamos el tiempo total
        t_medio=(t_fin-t_inicio)/(dataset.shape[0]-dataset_size) #Obtenemos el tiempo medio, restamos el número de filas total del dataset menos el anterior porque son los nuevos flujos que clasifica, inicialmente el anterior vale 1 (fila de características)

        n_bytes=dataset['Subflow Fwd Byts']+dataset['Subflow Bwd Byts']     #Sumamos la columna de enviados y recibidos para tener el columna con el total de bytes
        n_bytes_tot=n_bytes.sum()                                           #Obtenemos la suma de todas las filas para hallar el total de bytes 

        print("Se han clasificado",(dataset.shape[0]-dataset_size),"flujos en",t_medio) #Obtenemos el número de flujos clasificados por tiempo
        print("El tiempo total ha sido ",t_total, "segundos")                           #Vemos el tiempo total
        print("El ancho de banda en bytes/s es ",n_bytes_tot/t_total)                   #Obtenemos el ancho de banda dividiendo el número de bytes total entre el tiempo total
        print("El ancho de banda en bits/s es ",(n_bytes_tot*8)/t_total)                #Obtenemos el ancho de banda en bits/s
        dataset_size=dataset.shape[0]                                                   #Actualizamos el valor del número de filas 

#El bucle se ejecuta contantemente, se lee el archivo csv que genera el CICFLOWMETER, si el número de filas del dataset leído es mayor que 1
#entonces ya tiene un flujo (la primera fila indica las características). Y se lee constantemente el dataset viendo el núemero de filas. Si 
#el número de filas es mayor que el almacenado en dataset_size (lectura anterior) es que han llegado flujos nuevos y entonces clasificamos.

#Para calcular el tiempo medio se saca el tiempo antes y después de la clasificación y se hace una resta. Para calcularlo cada nueva llegada 
#de flujos, lo que hacemos es restar el número total de flujos menos el que había en la lectura anterior, obteniendo los nuevos flujos que 
#se van a clasificar.

263101
Se han clasificado 263100 flujos en 0.00011957751349866367
El tiempo total ha sido  31.460843801498413 segundos
El ancho de banda en bytes/s es  568365.9380791418
El ancho de banda en bits/s es  4546927.504633134
